In [ ]:
%gui qt6
# 주피터 환경에서 PyQt6 를 띄우기 위한 명령어
# 라이브러리
import sys  # 파이썬 시스템 조작 도구
import random  # 진동 효과를 위한 무작위 숫자 뽑기 위해
# 화면구성 위젯                       앱 : 기본창: 라벨(로고):  틀    : 투명도(흐릿하게)
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QFrame, QGraphicsOpacityEffect, \
                            QVBoxLayout, QPushButton,QStackedWidget,QGraphicsDropShadowEffect,QDialog
                 #  세로로 쌓아서 배치 : 버튼  : 여러화면을 꽂아두고 한장씩 보여줌, 그림자효과           , 대화상자
from PyQt6.QtCore import Qt, QPropertyAnimation, QRect, QTimer, QParallelAnimationGroup,QEasingCurve,QSequentialAnimationGroup,QPoint
    # 중앙정렬, 테두리X : 문이 양옆으로 열리고 로고가 스르륵 : 사각형 좌표 : 알람시계 : 왼쪽 문, 오른쪽 문, 로고 투명도를 동시에 움직이게 조율 , 속도감(점프)
    # QRect (Rectangle): 위치(x, y) + 크기(너비, 높이) , QPoint (Point): 오직 위치(x, y)만
    # QSequentialAnimationGroup 공이 가로 세로 이동한 뒤 꺽어지는 이동시 사용
from PyQt6.QtGui import QPixmap, QColor
            #로봇 이미지 파일 불러오기

# PyQt 앱을 실행하기 위한 QApplication 객체 생성 쥬피터 노트에서는 반드시 필요!
app = QApplication.instance()
if not app:
    app = QApplication(sys.argv)  # 이미 존재하면 재사용, 없으면 새로 만듦


# 로봇 얼굴 화면 (메인 1 화면)
class RobotMainScreen(QWidget):  #QWidget 상속받은 메인화면
    def __init__(self, parent=None):  # 생성자
        super().__init__(parent)  # 부모클래스 기능 그대로
        self.setFixedSize(800, 600)  # 창 크기를 800x600으로 고정
        self.setStyleSheet("background-color: #050505;")  # 창 전체 배경색을 칠해 빈틈 방지 다크 그레이색

        # 로봇의 귀여운 뿔 (안테나)
        self.ant_l_stick = QFrame(self)  # 직선형 기둥
        self.ant_l_stick.setGeometry(270, 50, 15, 60)
        # 위젯의 위치,크기(가로,세로,너비,높이)
        self.ant_l_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        # 위젯의 모든면에서 3픽셀 두께의 흰색 선을 그려라!, 테두리 중 아래만 없애라! , 배경은 투명
        self.ant_l_tip = QFrame(self)
        self.ant_l_tip.setGeometry(252, 35, 50, 25)
        self.ant_l_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #FF8033;")  # 주황색

        self.ant_r_stick = QFrame(self)
        self.ant_r_stick.setGeometry(515, 50, 15, 60)
        self.ant_r_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        self.ant_r_tip = QFrame(self)
        self.ant_r_tip.setGeometry(497, 35, 50, 25)
        self.ant_r_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #00A3E8;")  # 하늘색, 바다색

        # 로봇 얼굴 : face_outline
        self.face_outline = QFrame(self)
        self.face_outline.setGeometry(150, 110, 500, 330)
        self.face_outline.setStyleSheet("border: 4px solid #4364F7; border-radius: 60px; background-color: #111;")
        # 로열 블루 ,모서리 끝점에서 가로 세로 60px지점을 부드러운 원형 곡선으로 , #020202 보다는 살짝 더 밝은 검은색

        # 눈 버튼
        self.btn_manual = QPushButton("직접 이동", self.face_outline)
        self.btn_manual.setGeometry(60, 70, 160, 110)  # #1A1A1A 다크 그레이, #FF8033주황색, solid 는 실선, #FF8033주황색
        self.btn_manual.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #FF8033;
                font-size: 18px; font-weight: bold; border: 2px solid #FF8033; border-radius: 25px;
            }
            QPushButton:hover { background-color: #FF8033; color: black; }
        """)  # 마우스를 올릴때만 작동

        self.btn_auto = QPushButton("자동 이동", self.face_outline)
        self.btn_auto.setGeometry(280, 70, 160, 110)  # #00A3E8 맑고 깊은 파란색
        self.btn_auto.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #00A3E8;
                font-size: 18px; font-weight: bold; border: 2px solid #00A3E8; border-radius: 25px;
            }
            QPushButton:hover { background-color: #00A3E8; color: black; }
        """)

        # '=' 입 모양
        self.mouth_top = QFrame(self.face_outline)
        self.mouth_top.setGeometry(210, 240, 80, 5)
        self.mouth_top.setStyleSheet("background-color: #00FF00; border-radius: 2px;")  # #00FF00 형광연두색
        self.mouth_bottom = QFrame(self.face_outline)
        self.mouth_bottom.setGeometry(210, 255, 80, 5)
        self.mouth_bottom.setStyleSheet("background-color: #00FF00; border-radius: 2px;")

        # 하단 안내 칸
        self.bottom_bar = QFrame(self)
        self.bottom_bar.setGeometry(150, 500, 500, 70)
        self.bottom_bar.setStyleSheet("border: 2px solid white; background-color: #222; border-radius: 20px;")  #222 목탄색
        self.bottom_label = QLabel("이동 방식을 선택해주세요.", self.bottom_bar)
        self.bottom_label.setGeometry(0, 0, 500, 70)
        self.bottom_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 글자를 정중앙으로!!
        self.bottom_label.setStyleSheet("color: white; font-size: 18px; font-weight: bold; border: none;")


class ManualControlScreen(QWidget):  # 직접 이동 화면
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setFixedSize(800, 600)
        self.setStyleSheet("background-color: #0A0E14;")  # 블랙에 가까운 네이비

        # --- 스타일 시트 ---
        # 이동 버튼 (Cyan 네온)
        move_btn_style = """
            QPushButton {
                        /* 방사형그라데이션   ( 버튼 중앙에서 바깥으로 , 높을수록 퍼짐 , 0은 중심 약간 밝은 남색, 1은 바깥쪽 아주 어두운 남색*/
                background: qradialgradient(cx:0.5, cy:0.5, radius:0.5, stop:0 #2A3A4A, stop:1 #16212B);
                /* 2픽셓의 밝은 민트 실선   , 둥근 모서리         ,테두리와 같은 형광하늘색 */
                border: 2px solid #00F2FF; border-radius: 20px;color: #00F2FF; font-size: 35px; font-weight: bold;}
            /* 마우스를 대면            밝은 민트에서     아주 어두운 남색 */
            QPushButton:hover { background: #00F2FF; color: #0A0E14; border: 2px solid white; }
            /* 마우스를 클릭 하면    조금 더 어두운 청록색  */
            QPushButton:pressed { background: #008E9B; }
        """
        # 각도 버튼 (Orange 네온)
        #  이미 만들어 둔 디자인 틀을 그대로 쓰고         민트색 ->주황    ,  어두운 남색 -> 어두운 갈색     ,  진한 청록색  --> 진한 주황색으로
        angle_btn_style = move_btn_style.replace("#00F2FF", "#FF8033").replace("#2A3A4A", "#3D2B1F").replace("#008E9B","#CC5200")

        # --- 상단 제목 ---
        self.title_frame = QFrame(self)
        self.title_frame.setGeometry(60, 40, 200, 50)     # 진한 주황
        self.title_frame.setStyleSheet("background-color: #FF8033; border-radius: 12px; border: 2px solid white;")
        self.title_label = QLabel("직접 이동", self.title_frame)
        self.title_label.setGeometry(0, 0, 200, 50)
        self.title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 정중앙
        self.title_label.setStyleSheet("color: black; font-size: 18px; font-weight: bold; border: none;")

        # --- 메인으로 버튼 ---
        self.btn_back = QPushButton("메인으로", self)
        self.btn_back.setGeometry(650, 40, 110, 50)
        self.btn_back.setStyleSheet("""
            QPushButton {
                background: #1A1A1A; border: 3px solid #FF8033; border-radius: 10px;
                color: #FF8033; font-size: 16px; font-weight: bold;
            }
            QPushButton:hover { background: #FF8033; color: black; }
        """)    #  #1A1A1A 무채색 그레이

        # --- 위치 및 간격 설정 ---
        move_x = 110  # 왼쪽 전체 위치  'MOVE' 섹션에 속한 모든 요소의 시작점
        angle_x = 540  # 오른쪽 전체 위치
        btn_size = 90  # 버튼 크기
        gap = 20  # 버튼 사이의 여백

        # --- MOVE 섹션 ---
        self.move_tag = QLabel("▼ MOVE", self)
        self.move_tag.setGeometry(move_x, 180, 220, 30)
        self.move_tag.setStyleSheet("color: #00F2FF; font-weight: bold; font-size: 15px;")
                                           # 민트
        # 상단버튼 기준 간격 계산 배치
        self.btn_up = QPushButton("▲", self)
        self.btn_up.setGeometry(move_x + btn_size + gap, 230, btn_size, btn_size)
        self.btn_up.setStyleSheet(move_btn_style)
        self.btn_left = QPushButton("◀", self)
        self.btn_left.setGeometry(move_x, 230 + btn_size + gap, btn_size, btn_size)
        self.btn_left.setStyleSheet(move_btn_style)
        self.btn_right = QPushButton("▶", self)
        self.btn_right.setGeometry(move_x + (btn_size + gap) * 2, 230 + btn_size + gap, btn_size, btn_size)
        self.btn_right.setStyleSheet(move_btn_style)
        self.btn_down = QPushButton("▼", self)
        self.btn_down.setGeometry(move_x + btn_size + gap, 230 + (btn_size + gap) * 2, btn_size, btn_size)
        self.btn_down.setStyleSheet(move_btn_style)

        # --- ANGLE 섹션 ---
        self.angle_tag = QLabel("▼ ANGLE", self)
        self.angle_tag.setGeometry(angle_x, 180, 220, 30)
        self.angle_tag.setStyleSheet("color: #FF8033; font-weight: bold; font-size: 15px;")
        # 위 버튼
        self.btn_ang_up = QPushButton("▲", self)
        self.btn_ang_up.setGeometry(angle_x + 35, 230, btn_size, btn_size);
        self.btn_ang_up.setStyleSheet(angle_btn_style)
        # 아래 버튼
        self.btn_ang_down = QPushButton("▼", self)
        self.btn_ang_down.setGeometry(angle_x + 35, 230 + (btn_size + gap) * 2, btn_size, btn_size);
        self.btn_ang_down.setStyleSheet(angle_btn_style)


class AutoMapScreen(QWidget):   # 일부 변경
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setFixedSize(800, 600)
        self.setStyleSheet("background-color: #050505;")   # 리치 블랙

        # 상단 제목 (하늘색 상자)
        self.title_frame = QFrame(self)
        self.title_frame.setGeometry(30, 30, 180, 45)  # x 가 작을 수록 왼쪽
        self.title_frame.setStyleSheet("background-color: #00A3E8; border-radius: 10px; border: 2px solid white;")  # 딥스카이 블루
        self.title_label = QLabel("자동 이동", self.title_frame)
        self.title_label.setGeometry(0, 0, 180, 45)
        self.title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.title_label.setStyleSheet("color: black; font-size: 16px; font-weight: bold; border: none;")

        # "출발지를 선택해주세요." 안내 문구
        # 제목(x=30+180=210)과 메인으로 버튼(x=650) 사이 중앙에 배치
        self.guide_label = QLabel("출발지를 선택해주세요.", self)
        self.guide_label.setGeometry(210, 30, 390, 45) # x,y 너비,높이
        self.guide_label.setAlignment(Qt.AlignmentFlag.AlignCenter)                        # 1 픽셀씩 틈을 주시오!
        self.guide_label.setStyleSheet("color: #00A3E8; font-size: 18px; font-weight: bold;letter-spacing: 1px;")

        # 메인으로 버튼
        self.btn_back = QPushButton("메인으로", self)
        self.btn_back.setGeometry(650, 30, 110, 45)  #630
        self.btn_back.setStyleSheet("background: #1A1A1A; border: 2px solid #00A3E8; border-radius: 10px; color: #00A3E8; font-weight: bold;")                           # 무채색 그레이                    # 딥스카이블루

        # 버튼을 누르면 초기화 함수가 실행
        self.btn_back.clicked.connect(self.reset_screen)

        # 지도 데이터
        self.map_data = [
            [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
            [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
            [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
            [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
            [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
            [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
            [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
        ]

        # 전체 폭
        # offset 기준점으로부터의 거리(간격)
        self.offset_x = 40  # 작을 수록 왼쪽으로 시작점
        self.offset_y = 100  # 위쪽 여백 , 시작점
        self.cell_w = 56  # 가로 간격 커지면 옆으로
        self.cell_h = 70  # 세로 간격 커지면 아래로

        # 모든 공(버튼) 객체를 저장할 2차원 비어 있는 리스트 (7행 13열) 이게 없으면 계속 새 공으로 덮어씀.
        self.btns = [[None for _ in range(13)] for _ in range(7)]

        # ★ 로봇 배우 미리 준비 (처음엔 안 보이게)
        self.robot = QLabel(self)
        self.robot.hide()

        # 선택된 좌표를 저장할 공간
        self.start_pos = None  # 처음엔 비어있음 출발지 좌표 보관용
        self.end_pos = None

        self.draw_map()
    # 7 X 13
    def draw_map(self):
        for r in range(7): # 행(세로방향) 0~6 까지 반복
            for c in range(13): # 열(가로 방향) 0~12 까지 반복
                val = self.map_data[r][c] # 현재 칸의 데이터 가져오기!
                # 실제 화면에 그려질 좌표 계산
                # 시작점(offset) + (현재 칸 번호 * 간격)
                x = self.offset_x + (c * self.cell_w)
                y = self.offset_y + (r * self.cell_h)

                # 긴 벽
                # 양 끝 열(0번째,12번째열) 이면서 데이터가 1인 경우
                if (c == 0 or c == 12) and val == 1:
                    if r == 0 or r == 4:   # 0행 또는 4행에서만
                        wall = QFrame(self)  # 벽 모양 틀
                        # 벽의 틈 조정
                        wall.setGeometry(x + 5, y, 25, self.cell_h * 3 - 15)
                    # x는 왼쪽에서 +5 만큼,일반셀보다 얇게 그래야 벽처럼 보이니깐,3칸을 통째로 붙이는데 아래 닿지말라고 15 빼줌
                        wall.setStyleSheet("background-color: #222; border: 1px solid #444;")
                                                    #검은색에 가까운 진한 회색    ,  중간 단계의 어두운 회색
                    continue

                if val == 1:  # 네모 상자(선택불가)
                    box = QFrame(self)
                    box.setGeometry(x - 2, y -2, 45, 55)  # 네모 크기 조정 상자가 오른쪽벽에 너무 붙어서 -값 사용
                    box.setStyleSheet("background-color: #1a1a1a; border: 1px solid #555; border-radius: 5px;")
                                                       # 진한 숯색                 중간회색     0 완전 직사각형  5는 살짝 둥글게
                                                                                 # 값이 커질수록 둥글어지고 원래크기의 절반이면 원형
                elif val == 2:  # 하늘색 공 (선택 가능)
                    btn = QPushButton(self)  # 클릭 버튼
                    # 위치 크기 조절
                    btn.setGeometry(x + 10, y + 12, 26, 26)
                    # 초기 하늘색 스타일 공
                    btn.setStyleSheet("""  /*  투명 배경                          하늘색    크기가 26이니깐 13이면 원형 */
                        QPushButton { background: transparent; border: 2px solid #00ffff; border-radius: 13px; }
                        QPushButton:hover { background: rgba(0, 255, 255, 0.6); }
                    """)   # 마우스를 올리면                    밝아지게      투명도
                    # 클릭시 실행될 함수 연결 (현재 행, 열 전달)
                    btn.clicked.connect(lambda ch, row=r, col=c: self.on_ball_clicked(row, col))

                    # 리스트에 버튼 저장
                    self.btns[r][c] = btn

    def on_ball_clicked(self, row, col):   # 공 클릭시 공들이 주황색으로 변하는 함수
        # 출발지 선택
        if self.start_pos is None:
            self.start_pos = (row, col) # 현재 좌표를 출발지로 저장

            # 안내 문구 변경 (출발지 --> 도착지)
            self.guide_label.setText("도착지를 선택해주세요.")
            self.guide_label.setStyleSheet("color: #FF8033; font-size: 18px; font-weight: bold;")

            # 스타일 설정
            orange_border = "background: transparent; border: 2px solid #FF8033; border-radius: 13px;"
            sky_blue_fill = "background: #00A3E8; border: 2px solid white; border-radius: 13px;"

            for r in range(7):
                for c in range(13):
                    if self.btns[r][c]:  # 칸에 버튼이 있다면
                        if (r, c) == self.start_pos:
                            self.btns[r][c].setStyleSheet(sky_blue_fill) # 출발지는 파랑!
                        else:
                            self.btns[r][c].setStyleSheet(orange_border) # 나머지는 주황 테두리!
            print(f"출발지 선택 완료: {self.start_pos}")

        # 출발지가 이미 있고, 이제 도착지 선택
        else:
            if (row, col) == self.start_pos: # 출발지를 또 누르면 무시
                return
            # 클릭한 좌표를 end_pos에 꼭! 저장
            self.end_pos = (row, col) # ★ 이 줄이 빠져있거나 위치가 중요해요!

            # 도착지 강조 스타일 (진한 주황색 + 흰색 테두리)
            final_target_style = "background: #CC5200; border: 2px solid white; border-radius: 13px;"
            self.btns[row][col].setStyleSheet(final_target_style)

            # 최종 안내 문구
            self.guide_label.setText(f"목적지 확정! ({row}, {col})로 이동 시작.")
            self.guide_label.setStyleSheet("color: #CC5200; font-size: 20px; font-weight: bold;")

            # ★ 여기서 로봇 연출 함수를 실행합니다!
            self.start_robot_animation(self.start_pos, self.end_pos)

            print(f"최종 목적지 확정: {row}, {col}")
            # 여기서 이동 애니메이션 함수를 호출하면 완벽합니다!

    def start_robot_animation(self, start, end):
        # 목적지가 없으면 실행하지마라!
        if end is None: return

        # 출발지(start)와 목적지(end) 공만 남기고 다 숨기기
        for r in range(7):
            for c in range(13):
                if self.btns[r][c] and (r, c) != start and (r, c) != end:
                    self.btns[r][c].hide()

       # 로봇 준비
        pixmap = QPixmap("이동로봇.png")
        self.robot.setPixmap(pixmap) # 여기서 scaled를 미리 하지 말고.
        self.robot.setScaledContents(True) # 라벨 크기가 줄어들면 사진도 같이 줄어들게!

        # png 투명도 살리는 핵심 설정
        self.robot.setAttribute(Qt.WidgetAttribute.WA_TranslucentBackground)
        self.robot.setStyleSheet("background:transparent; border:none;") # 잔여 배경 제거

        # 처음 등장할 때 위치와 크기 (200x200)
        self.robot.setGeometry(300, 50, 200, 200)   # 시작위치 중앙위쪽
        self.robot.show() # 이제 보여줌
        self.robot.raise_()  # 로봇이 다른 그림들 보다 위에 오개!

        # 출발지 실제 좌표 계산 (작아졌을 때 갈 곳)
        # 공의 크기가 26x26이므로, 로봇(40x40)을 공 정중앙에 올리기 위한 보정값(-7)
        start_x = self.offset_x + (start[1] * self.cell_w) + 10-7
        start_y = self.offset_y + (start[0] * self.cell_h) + 12-7  #15

        # 등장 애니메이션: 중앙(크게) -> 출발지(작게)
        self.intro_ani = QPropertyAnimation(self.robot, b"geometry")
        self.intro_ani.setDuration(3000) # 2초 동안 실행 조금 더 부드럽게
        self.intro_ani.setStartValue(QRect(300, 50, 200, 200)) # 시작 위치/크기
        self.intro_ani.setEndValue(QRect(start_x, start_y, 40, 40)) # 출발지 위치
        self.intro_ani.setEasingCurve(QEasingCurve.Type.OutQuart) # 실선 방지용 부드러운 곡선

        # ★ 핵심: "등장"이 끝나면 "점프" 함수를 실행하라는 약속!
        self.intro_ani.finished.connect(lambda: self.start_jumping(start, end))
        self.intro_ani.start()
        # 안내 문구 변경
        self.guide_label.setText("로봇이 준비 중입니다...")

    def start_jumping(self, start_pos, end_pos):
        # 안내 문구 변경
        self.guide_label.setText("통로를 따라 이동중...")
        self.guide_label.setStyleSheet("color: #FF8033; font-size: 20px; font-weight: bold;")

        # 길 찾기 (BFS) - 벽과 상자를 피해서 이동 가능!
        def find_path(start, end):
            # 'wait_list'에 현재 경로를 담아 줄을 세움!
            wait_list = [[start]]
            checked = [start] # 이미 확인한 좌표들

            while wait_list:
                # 가장 먼저 줄 서있던 경로를 꺼냄, (pop 0)
                current_path = wait_list.pop(0)
                current_spot = current_path[-1] # 현재 로봇이 있는 위치

                if current_spot == end: # 도착지 발견!
                    return current_path

                # 주변(상, 하, 좌, 우)에 갈 수 있는 길이 있는지 훑어봄!
                r, c = current_spot
                # 세로(1,0)를 먼저 넣어서 세로 이동을 우선시함
                for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    next_r, next_c = r + dr, c + dc

                    # 지도 안이고, 길(2)이고, 아직 확인 안 한 곳이라면?
                    if 0 <= next_r < 7 and 0 <= next_c < 13:
                        if self.map_data[next_r][next_c] == 2 and (next_r, next_c) not in checked:
                            checked.append((next_r, next_c)) # 확인 완료 표시

                            # 새로운 경로를 만들어서 줄 맨 뒤에 세움!
                            new_path = list(current_path)
                            new_path.append((next_r, next_c))
                            wait_list.append(new_path)
            return None

        # 실제 이동 경로 계산
        real_path = find_path(start_pos, end_pos)

        if not real_path:
            self.guide_label.setText("경로 못 찾겠다!")
            return

        # 로봇 이동 애니메이션 설정
        self.move_group = QSequentialAnimationGroup()
        prev_r, prev_c = start_pos   # 현재 위치 추적을 위한 변수

        # 찾은 경로(real_path)를 따라 한 칸씩 이동
        for next_r, next_c in real_path[1:]:
            # 시작/끝 픽셀 좌표 계산 (-7 보정값 적용)
            s_x = self.offset_x + (prev_c * self.cell_w) + 10 - 7
            s_y = self.offset_y + (prev_r * self.cell_h) + 12 - 7
            e_x = self.offset_x + (next_c * self.cell_w) + 10 - 7
            e_y = self.offset_y + (next_r * self.cell_h) + 12 - 7
            # 한 칸 이동 애니메이션
            step_ani = QPropertyAnimation(self.robot, b"geometry")
            step_ani.setDuration(300) # 한 칸 이동 시간 (0.3초)
            step_ani.setStartValue(QRect(s_x, s_y, 40, 40))
            step_ani.setEndValue(QRect(e_x, e_y, 40, 40))
            step_ani.setEasingCurve(QEasingCurve.Type.InOutSine)

            # 그룹에 추가
            self.move_group.addAnimation(step_ani)
            self.move_group.addPause(50) # 칸마다 살짝 멈추는 로봇 0.05초 휴식!
            # 이전 위치 업데이트
            prev_r, prev_c = next_r, next_c

        # 드디어 출발!
        self.move_group.finished.connect(self.on_arrival)
        self.move_group.start()

    def on_arrival(self):
        self.guide_label.setText("도착 완료! 임무 성공")
        self.guide_label.setStyleSheet("color: #00FF00; font-size: 22px; font-weight: bold;")

        # [도착 세리머니] 로봇을 위로 살짝 튕기기!
        curr_rect = self.robot.geometry()
        self.jump_ani = QPropertyAnimation(self.robot, b"geometry")
        self.jump_ani.setDuration(500) # 점프 한 번에 걸리는 시간
        self.jump_ani.setStartValue(curr_rect)
        # 시작점 (바닥)
        self.jump_ani.setStartValue(curr_rect)
        # 중간점 (공중 30픽셀 위로!)
        self.jump_ani.setKeyValueAt(0.5, QRect(curr_rect.x(), curr_rect.y() - 30, 40, 40))
        # 끝점 (다시 바닥)
        self.jump_ani.setEndValue(curr_rect)
        # 3번 반복하라!
        self.jump_ani.setLoopCount(3)
        self.jump_ani.setEasingCurve(QEasingCurve.Type.OutCubic) # 점프할 때 더 자연스럽게!
        self.jump_ani.start()

        # 점프가 다 끝나면 메시지 박스 띄우기
        # 점프 애니메이션이 끝난 직후에 창이 뜨도록 연결
        self.jump_ani.finished.connect(self.show_finish_dialog)

    def show_finish_dialog(self):
        dlg = MyFinishDialog(self)  # 실제 실행 하시오!
        if dlg.exec(): # 버튼 느릴때!
            # 확인을 눌렀을 때 다음 화면으로 이동!
            self.go_to_next_screen()


    def reset_screen(self):
        # 저장된 좌표 데이터 초기화
        self.start_pos = None
        # 로봇도 숨기기
        self.robot.hide()

        # 안내 문구와 제목 스타일을 처음 하늘색 상태로!
        self.guide_label.setText("출발지를 선택해주세요.")
        self.guide_label.setStyleSheet("color: #00A3E8; font-size: 18px; font-weight: bold; letter-spacing: 1px;")
        #self.title_frame.setStyleSheet("background-color: #00A3E8; border-radius: 10px; border: 2px solid white;")

        # 모든 공의 색깔을 다시 원래(기본값)의 투명한 하늘색 테두리로!
        default_style = "background: transparent; border: 2px solid #00ffff; border-radius: 13px;"
        for r in range(7):
            for c in range(13):
                if self.btns[r][c]: # 버튼이 있는 칸만 골라서
                    self.btns[r][c].show() # ★ 숨겼던 공들을 다시 보여주기!
                    self.btns[r][c].setStyleSheet(default_style)

        print("화면이 처음 상태로 리셋되었습니다!")

class MyFinishDialog(QDialog):  # QDialog 대화창의 모든 기능
    def __init__(self, parent=None):
        super().__init__(parent)
        # 기존의 플래그들을 유지하면서 상단바만 없애는 방식 (| 연산자 사용) | 안쓰면 앞에 인트로 문열리는거 사라짐.
        self.setWindowFlags(self.windowFlags() | Qt.WindowType.FramelessWindowHint)
        self.setFixedSize(450, 220) # 창 크기
        # 맵 배경과 어울리는 어두운 배경색          다크그레이                    하늘색
        self.setStyleSheet("background-color: #2b2b2b; border: 2px solid #00AAFF; border-radius: 12px;")

        layout = QVBoxLayout() # 레이아웃 박스
        layout.setContentsMargins(20, 30, 20, 30) # 여백 조절 (왼,위,오,아래)

        # 메인 글자 설정                                                                              바깥쪽 여백   -는 아래로
        self.label = QLabel('목적지에 무사히 도착! <img src="이동로봇.png" width="45" height="45" style="margin-bottom: -15px;>')
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        # 글자 크기를 키우고 하얗게 설정 (border: none은 상속 방지용)
        self.label.setStyleSheet("color: white; font-size: 32px; font-weight: bold; border: none;")

        # 치우친 그림자 효과 적용
        shadow = QGraphicsDropShadowEffect()
        shadow.setBlurRadius(0)          # 그림자를 뭉개지 않고 선명하게
        shadow.setOffset(5, 5)           # 오른쪽 아래로 5px 치우치게! x축은 + 오른쪽 , y 축은 + 아래쪽
        shadow.setColor(QColor(0, 0, 0)) # 완전 검은색 그림자
        self.label.setGraphicsEffect(shadow)

        # 확인 버튼 설정
        self.ok_btn = QPushButton("확인")
        self.ok_btn.setFixedSize(140, 50)
        self.ok_btn.setStyleSheet("""
            QPushButton {
                background-color: #444444;  /* #000000완전 검정<#2b2b2b<회색<#aaaaaa<#ffffff 완전 화이트
                color: white;
                border: 2px solid #00AAFF;
                border-radius: 8px;
                font-size: 20px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #00AAFF;
                color: black;
            }
        """)
        # 확인 버튼 누르면 창 닫기
        self.ok_btn.clicked.connect(self.accept)

        # 레이아웃에 배치
        layout.addWidget(self.label) # 글자 맨위
        layout.addSpacing(20) # 글자와 버튼 사이 간격 20 비우공
        layout.addWidget(self.ok_btn, alignment=Qt.AlignmentFlag.AlignCenter) # 확인버튼은 센터로!
        self.setLayout(layout) # 보여줘!


# 대장 클래스 (전체 배경색 적용)
class LegosApp(QWidget):  # 전체 GUI 담당
    def __init__(self):
        super().__init__()
        self.setFixedSize(800, 600)  # 가로,세로 테두리 없는 창
        self.setWindowFlags(Qt.WindowType.FramelessWindowHint)  # 위쪽의 닫기/최소화바 없앰.
        self.setStyleSheet("background-color: #050505;")  # 전체 앱 배경을 검은색으로 고정하여 테두리 빈틈 방지

        # 슬라이드 박스 생성
        self.layout = QVBoxLayout(self)
        self.layout.setContentsMargins(0, 0, 0, 0)
        self.stack = QStackedWidget()  # 화면 쌓기
        self.layout.addWidget(self.stack)

        # 화면 등록
        self.main_view = RobotMainScreen()  # Index 0: 로봇 얼굴
        self.stack.addWidget(self.main_view)

        self.manual_view = ManualControlScreen()  # Index 1: 직접 이동
        self.stack.addWidget(self.manual_view)

        self.map_view = AutoMapScreen()  # Index 2: 자동 이동
        self.stack.addWidget(self.map_view)

        # 버튼과 인덱스 번호 맞추기
        # 직접 이동 버튼 -> Index 1번 화면으로
        self.main_view.btn_manual.clicked.connect(lambda: self.stack.setCurrentIndex(1))
        # 클릭시 stack 에서 1번화면을 꺼내줘!   # lambda 사용이용 :  클릭하는 순간 실행되도록 하기 위해!

        # 자동 이동 버튼 -> Index 2번 화면으로
        self.main_view.btn_auto.clicked.connect(lambda: self.stack.setCurrentIndex(2))

        # 각 화면의 '메인으로' 버튼 -> Index 0번(얼굴)으로
        self.manual_view.btn_back.clicked.connect(lambda: self.stack.setCurrentIndex(0))
        self.map_view.btn_back.clicked.connect(lambda: self.stack.setCurrentIndex(0))

        # 인트로 레이어
        self.intro_layer = QFrame(self)
        # 인트로 레이어 생성 (항상 맨 마지막에 만들어야 모든 화면을 덮는다! 딴거보다 먼저 만들면 엉뚱한 화면이 덮을 수도 있다.)
        self.intro_layer.setGeometry(0, 0, 800, 600)
        self.intro_layer.setStyleSheet("background-color: #050505;")  # 완전 검은색

        # 로고 설정 (인트로 레이어 소속)
        self.logo = QLabel("LEGOS", self.intro_layer)
        self.logo.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 정중앙
        self.logo.setGeometry(0, 225, 800, 150)  # 화면 중앙 높이쯤 배치
        self.current_spacing = -60  # 처음에 글자 뭉치게 자간 -60
        self.update_logo_style()  # 아래의 정의된 스타일 함수 호출
        # 로고가 '스르륵' 나타나게 해주는 투명도 필름(Effect) 장착
        self.logo_op = QGraphicsOpacityEffect(self.logo)
        self.logo.setGraphicsEffect(self.logo_op)
        self.logo_op.setOpacity(0)  # 시작은 투명 0 (안보임)
        # 문 2개 설정 (인트로 레이어 소속)
        self.left_door = QFrame(self.intro_layer)
        self.right_door = QFrame(self.intro_layer)
        self.setup_full_design_doors()  # 디자인 (못,손잡이, 경고띠) 입힘
        # 문 흔들기 애니메이션 시작 진동을 제어할 변수와 타이머 준비
        self.shake_count = 0  # 몇번 진동 했는지 기록
        self.shake_timer = QTimer(self)  # 째깍거리는 시계
        self.shake_timer.timeout.connect(self.do_shake)  # 시계가 울릴때마다 do_shake 실행 무작위로 x,y 이동! 진동 효과
        QTimer.singleShot(1000, self.start_shaking)  # 1초 후에 start_shaking 실행

    def update_logo_style(self):  # 로고 스타일

        self.logo.setStyleSheet(
            #글자 색상 : 선형그래디언트   시작점(0,0)위, 끝점(0,1)아래,위쪽끝:연한파랑,중간색 : 진한 파랑, 아래끝:짙은 파랑 ,   크기   :   150
            f"color: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #6FB1FC, stop:0.5 #4364F7, stop:1 #0052D4); font-size: 150px; font-weight: bold; letter-spacing: {self.current_spacing}px; background: transparent;")
                      # 굵게 ,   글자 간격       -60~15  널게 벌어짐       , 글자배경 :  투명 배경없음,배경색 그대로

    def setup_full_design_doors(self):  # 문 디테일
        # 오리지널 CSS 스타일 그대로 적용 CSS 코드들 변수에 저장(가독성위해)
        door_base_css = "background: #111; border: 1px solid #222;"  # 문 바디
                  # 배경 : 진한 검정색  , 테두리 1픽셀 두께 , 어두운 회색
        #패널 문안쪽 장식판 # 배경                   왼쪽   위   , 오른쪽 아래   대각선 방향으로 밝은회색->짙은회색     테두리 : 2픽셀, 회색
        panel_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #2a2a2a, stop:1 #151515); border: 2px solid #3d3d3d; border-radius: 4px;"  # 모서리 살짝 둥글게
        # 못(리벳)    # 배경       원형그라디언트       중심         , 반지름       중심색:연회색, 바깥색 : 검정  , 동그란 못 모양
        rivet_css = "background: qradialgradient(cx:0.4, cy:0.4, radius:0.7, stop:0 #ccc, stop:1 #111); border-radius: 7px;"
        # 손잡이  (메탈 느낌)                        왼쪽       , 오른쪽      , 왼쪽끝  검정   중앙     흰색   오른쪽끝 검정  ,  어두운 테두리
        handle_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:0, stop:0 #000, stop:0.5 #fff, stop:1 #000); border: 1px solid #444;"
        # 경고띠                      선형 그라디언트     반복       ,           반복되는 선 비율조정           노랑
        hazard_css = "background: qlineargradient(spread:reflect, x1:0, y1:0, x2:0.4, y2:0.1, stop:0 #fbc02d, stop:0.5 #fbc02d, stop:0.51 #000, stop:1 #000);"  # 검정 교차

        for door, side in [(self.left_door, "L"), (self.right_door, "R")]:  # L,R 구분위해 side 사용
            # 위젯 위치와 크기 지정                          너비,높이 (문크기)
            door.setGeometry(0 if side == "L" else 400, 0, 400, 600)
                          # 왼쪽(0,0,400,600) , 오른쪽 (400,0,400,600)
            door.setStyleSheet(door_base_css)  # "background: #111; 기본값(border: 1px solid #222;"검정, 1픽셀의 짙은회색)

            # 1. 직사각형 패널
            panel = QFrame(door)  # 사각형 빈틀
            panel.setGeometry(80, 100, 240, 400)
            # 위치 왼쪽,오른쪽 숫자만큼 떨어진,크기
            panel.setStyleSheet(panel_css)

            # 2. 못 (리벳)
            pts = [(15, 15), (210, 15), (15, 370), (210, 370)]  # 못 위치
            for px, py in pts:
                rv = QFrame(panel)
                rv.setGeometry(px, py, 14, 14)  # 크기
                rv.setStyleSheet(rivet_css)

            # 3. 메탈 손잡이
            h = QFrame(door)
            h.setGeometry(385 if side == "L" else 0, 250, 15, 100)
            # 위치  :문 오른쪽 끝       ,  문 왼쪽 끝 근처
            h.setStyleSheet(handle_css)

            # 4. 공장 무늬 (경고띠)
            haz = QFrame(door)
            haz.setGeometry(0, 560, 400, 40)
                    # 위치  문 하단  , 너비 , 높이
            haz.setStyleSheet(hazard_css)

    def start_shaking(self):
        self.shake_timer.start(15)  # 0.015 초마다 진동 알람.

    def do_shake(self):  # 알람이 울릴 때마다 실행되는 함수
        power = 10 + (self.shake_count // 20)
        # 강도 기본 10  + 20번 마다 1px 증가
        dx, dy = random.randint(-power, power), random.randint(-power, power)  # 랜덤 가로, 세로 값
        self.left_door.move(dx, dy)
        self.right_door.move(400 + dx, dy)  # 문을 랜덤위치로~
        self.shake_count += 1
        if self.shake_count > 80:  # 80번까지 진동!
            self.shake_timer.stop()  # 알람 stop
            self.left_door.move(0, 0)
            self.right_door.move(400, 0)  # 문 위치 다시 0으로 정렬
            self.start_opening()  # 문열기

    def start_opening(self):
        self.group = QParallelAnimationGroup()  # 동시에 여러 애니메이션 돌리기 위한 grop
        # 2.4초 동안 문을 화면 밖으로 완전히 밀어냄
        # QPropertyAnimation PyQt에서 위젯 속성을 시간에 따라 애니메이션 하는 클래스
        # PyQt의 QPropertyAnimation은 속성 이름을 문자열이 아닌 바이트 문자열로 받도록 되어 있음 그래서 b. 이렇게 씀
        # geometry = QRect(x, y, width, height)
        a1 = QPropertyAnimation(self.left_door, b"geometry");a1.setDuration(2400);a1.setEndValue(QRect(-400, 0, 400, 600))
             # 애니메이션 적용 위젯, 속성(위치, 크기)                       2.4초 움직임  ,             완쪽 화면 밖 , 문크기 유지
        a2 = QPropertyAnimation(self.right_door, b"geometry");a2.setDuration(2400);a2.setEndValue(QRect(800, 0, 400, 600))
        a3 = QPropertyAnimation(self.logo_op, b"opacity");a3.setDuration(2500);a3.setEndValue(1)
         # QLabel로고에 적용한 투명도 효과,속성                                     투명도 1 완전히 보임 문이 열리는 즉시 로고 보임.

        self.group.addAnimation(a1);self.group.addAnimation(a2);self.group.addAnimation(a3)
        self.group.start()  # 3개의 애니메이션 동시 시작!

        # 글자 자간을 벌리는 타이머도 별도로 작동
        self.st = QTimer(self)
        self.st.timeout.connect(self.anim_text)
        self.st.start(30)

    def anim_text(self):
        if self.current_spacing < 15:  # 자간이 15 될 때까지
            self.current_spacing += 1.0
            self.update_logo_style()  # 1픽셀씩 벌리고 로고 스타일 업데이트
        else:
            self.st.stop()  # 다 벌어지면 stop
            # 연출 감상을 위해 1.8초 대기 후 인트로 종료 함수 실행
            QTimer.singleShot(1800, self.finish_intro)

    def finish_intro(self):
        # 실행 되는 순간 인트로 레이어를 삭제하며 밑에 있던 메인 화면이 매끄럽게 보임
        self.intro_layer.deleteLater()

    def keyPressEvent(self, event):
        if event.key() == Qt.Key.Key_Escape: self.close()  # esc 누르면 꺼짐.


# 실행
main_window = LegosApp()  # 생성자 실행 되며 세팅
main_window.show()  # 보여줘!  # 로봇이 도착 후 3번 뜀!!! 그리고 상단바 없는 목적지 도착 확인 버튼 나옴!